In [2]:
#coding:utf-8
import torch as t
from torch import nn
embedding = t.nn.Embedding(10,2) # 十个词，每个词用2维词向量表示
input = t.arange(0,6).view(3,2).long() # 三个句子，每句有两个词
input = t.autograd.Variable(input)
output = embedding(input)
print output.size()
print embedding.weight.size()

torch.Size([3, 2, 2])
torch.Size([10, 2])


In [5]:
from torch.autograd import Variable
#输入词用10维词向量表示
#隐藏元用20维词向量表示
#两层的lstm
rnn = nn.LSTM(10,20,2)
#输入每句话有五个词，每个词由十维的词向量表示，总共有三句话（batchsize）
input =Variable(t.randn(5,3,10))
#隐藏元hidden_state 和cell_state的初始值
#形状 num_layers，batch_size, hidden_size
h0 = Variable(t.zeros(2,3,20))
c0 = Variable(t.zeros(2,3,20))
#output是最后一层所有隐藏元的值
#hn和cn是所有层的最后一个隐藏元的值
output, (hn, cn) = rnn(input, (h0,c0))

print output.size()
print (hn.size())
print cn.size()

torch.Size([5, 3, 20])
torch.Size([2, 3, 20])
torch.Size([2, 3, 20])


In [6]:
#对文字的预处理分为以下几步：
#中文分词
#将词用序号表示（word2idx），并过滤低频词
#将所有描述补齐到等长（pad_sequence）
#利用pack_padded_sequence进行计算加速
from torch.nn.utils.rnn import pack_padded_sequence,pad_packed_sequence
#dummydata
sen1 = [1,1,1]
sen2=[2,2,2,2]
sen3=[3,3,3,3,3,3,3,3,3,3,3,3]
sen4=[4,4,4,4,4]
sentences = [sen1,sen2,sen3,sen4]
sentences = sorted(sentences,key = lambda x:len(x),reverse=True)

In [7]:
lengths= [5 if len(sen) > 5 else len(sen) for sen in sentences]
lengths

[5, 5, 4, 3]

In [8]:
def pad_sen(sen,length=5,padded_num=0):
    origin_len = len(sen)
    padded_sen = sen[:length]
    padded_sen = padded_sen + [padded_num for _ in range(origin_len,length)]
    return padded_sen

In [9]:
pad_sentences = [pad_sen(sen) for sen in sentences]
pad_sentences

[[3, 3, 3, 3, 3], [4, 4, 4, 4, 4], [2, 2, 2, 2, 0], [1, 1, 1, 0, 0]]

In [12]:
#4*5 batch_size = 4  词=5
pad_tensor = t.Tensor(pad_sentences).long()

#5*4
pad_tensor = pad_tensor.t()
pad_variable = t.autograd.Variable(pad_tensor)
pad_variable #一列是一句话

Variable containing:
 3  4  2  1
 3  4  2  1
 3  4  2  1
 3  4  2  0
 3  4  0  0
[torch.LongTensor of size 5x4]

In [13]:
embedding = nn.Embedding(5,2) #五个词 每个词2维
pad_embeddings = embedding(pad_variable)
pad_embeddings

Variable containing:
(0 ,.,.) = 
  2.5003  0.4281
 -0.5557  2.0929
 -0.2063  1.1136
  0.3661 -0.9053

(1 ,.,.) = 
  2.5003  0.4281
 -0.5557  2.0929
 -0.2063  1.1136
  0.3661 -0.9053

(2 ,.,.) = 
  2.5003  0.4281
 -0.5557  2.0929
 -0.2063  1.1136
  0.3661 -0.9053

(3 ,.,.) = 
  2.5003  0.4281
 -0.5557  2.0929
 -0.2063  1.1136
 -1.3850 -0.7451

(4 ,.,.) = 
  2.5003  0.4281
 -0.5557  2.0929
 -1.3850 -0.7451
 -1.3850 -0.7451
[torch.FloatTensor of size 5x4x2]

In [14]:
#pack数据
packed_variable = pack_padded_sequence(pad_embeddings,lengths)
packed_variable

PackedSequence(data=Variable containing:
 2.5003  0.4281
-0.5557  2.0929
-0.2063  1.1136
 0.3661 -0.9053
 2.5003  0.4281
-0.5557  2.0929
-0.2063  1.1136
 0.3661 -0.9053
 2.5003  0.4281
-0.5557  2.0929
-0.2063  1.1136
 0.3661 -0.9053
 2.5003  0.4281
-0.5557  2.0929
-0.2063  1.1136
 2.5003  0.4281
-0.5557  2.0929
[torch.FloatTensor of size 17x2]
, batch_sizes=[4, 4, 4, 3, 2])

In [15]:
rnn = t.nn.LSTM(2,3) #2词向量，3隐藏元
output, hn = rnn(packed_variable)
output = pad_packed_sequence(output)
output

(Variable containing:
 (0 ,.,.) = 
   0.1774  0.5673  0.2668
  -0.0178 -0.0922  0.0357
   0.0001  0.0365  0.0390
   0.1024  0.2378  0.0083
 
 (1 ,.,.) = 
   0.2635  0.7143  0.4647
  -0.0245 -0.1127  0.0401
  -0.0043  0.0572  0.0532
   0.1229  0.3438  0.0320
 
 (2 ,.,.) = 
   0.2939  0.7590  0.5632
  -0.0272 -0.1185  0.0404
  -0.0081  0.0665  0.0583
   0.1192  0.3930  0.0538
 
 (3 ,.,.) = 
   0.3050  0.7753  0.6091
  -0.0282 -0.1204  0.0402
  -0.0105  0.0701  0.0601
   0.0000  0.0000  0.0000
 
 (4 ,.,.) = 
   0.3091  0.7818  0.6318
  -0.0286 -0.1211  0.0402
   0.0000  0.0000  0.0000
   0.0000  0.0000  0.0000
 [torch.FloatTensor of size 5x4x3], [5, 5, 4, 3])

In [21]:
#图像数据处理，修改最后一层
#第一种做法  修改forword函数
from torchvision.models import resnet50
def new_forward (self, x):
    x = self.conv1(x)
    #省略中间
    x = self.avgpool(x)
    x = x.vioew(x.size(0), -1)
    #x = self.fc(x)
    return x
model = resnet50(pretrained=True)
#覆盖
model.forward = lambda x:new_forward(model,x)
#或者 model.__class__.forward = new_forward

In [22]:
#第二种做法  删除全连接层 改成恒等映射
model = resnet50(pretrained = True)
del model.fc
model.fc = lambda x:x
model.cuda()